In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel("data/COVID_19.xlsx")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3308 entries, 0 to 3307
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Date time                              3308 non-null   datetime64[ns]
 1   Age                                    3303 non-null   object        
 2   Gender                                 3297 non-null   object        
 3   Region                                 3286 non-null   object        
 4   Do you smoke?                          3269 non-null   object        
 5   Have you had Covid`19 this year?       3305 non-null   object        
 6   IgM level                              3232 non-null   object        
 7   IgG level                              3248 non-null   object        
 8   Blood group                            3252 non-null   float64       
 9   Do you vaccinated influenza?           3294 non-null   object  

In [4]:
df = df.rename(columns={
    "Date time": "date_time",
    "Age": "age",
    "Gender": "gender",
    "Region": "region",
    "Do you smoke?": "is_smoker",
    "Have you had Covid`19 this year?": "had_covid_this_year",
    "IgM level": "igm_level",
    "IgG level": "igg_level",
    "Blood group": "blood_group",
    "Do you vaccinated influenza?": "is_vaccinated_influenza",
    "Do you vaccinated tuberculosis?": "is_vaccinated_tuberculosis",
    "Have you had influenza this year?": "had_influenza_this_year",
    "Have you had tuberculosis this year? ": "had_tuberculosis_this_year",
    "Maximum body temperature": "max_body_temp",
})

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3308 entries, 0 to 3307
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date_time                   3308 non-null   datetime64[ns]
 1   age                         3303 non-null   object        
 2   gender                      3297 non-null   object        
 3   region                      3286 non-null   object        
 4   is_smoker                   3269 non-null   object        
 5   had_covid_this_year         3305 non-null   object        
 6   igm_level                   3232 non-null   object        
 7   igg_level                   3248 non-null   object        
 8   blood_group                 3252 non-null   float64       
 9   is_vaccinated_influenza     3294 non-null   object        
 10  is_vaccinated_tuberculosis  3303 non-null   object        
 11  had_influenza_this_year     3299 non-null   object      

Let's check whether "Date time" is good to become an index column:

In [6]:
date_time_duplicates_cnt = df["date_time"].duplicated(keep="first").sum()

print(f"\"Date time\" duplicates found: {date_time_duplicates_cnt}")
print(f"Thats's {date_time_duplicates_cnt / len(df) * 100}%")

"Date time" duplicates found: 1246


Thats's 37.66626360338573%


As we can see, the "Date time" column has a bunch of duplicates: more than 1/3!
That's why we shouldn't use this column as an index

In [7]:
# DEMO ONLY!
df.set_index("date_time").head(4)

,age,gender,region,is_smoker,had_covid_this_year,igm_level,igg_level,blood_group,is_vaccinated_influenza,is_vaccinated_tuberculosis,had_influenza_this_year,had_tuberculosis_this_year,max_body_temp
date_time,,,,,,,,,,,,,
2020-09-20 11:23:15,40-65,Female (Жінка),"Ukraine, Lviv (Львів)",No,Maybe (можливо),<0.9 (negative/негативний),0.9-1.1 (indefinable/невизначений),2.0,Yes,Yes,No,No,38.1
2020-09-20 11:23:15,23-40,Female (Жінка),"Ukraine, Chernivtsi",No,Yes,>1.1 (positive/позитивний),0.9-1.1 (indefinable/невизначений),2.0,No,Yes,No,No,37.0
2020-09-20 11:23:15,23-40,Female (Жінка),"Ukraine, Lviv (Львів)",No,Maybe (можливо),<0.9 (negative/негативний),<0.9 (negative/негативний),2.0,No,Yes,Yes,No,37.8
2020-09-20 11:23:15,40-65,Male (Чоловік),"Ukraine, Chernivtsi",No,No,<0.9 (negative/негативний),<0.9 (negative/негативний),2.0,No,Yes,No,No,NaN


In [8]:
df.isnull().sum()

date_time                        0
age                              5
gender                          11
region                          22
is_smoker                       39
had_covid_this_year              3
igm_level                       76
igg_level                       60
blood_group                     56
is_vaccinated_influenza         14
is_vaccinated_tuberculosis       5
had_influenza_this_year          9
had_tuberculosis_this_year       9
max_body_temp                 1775
dtype: int64

In [9]:
# Delete columns not related to the research
df = df.drop(columns=["is_smoker", "is_vaccinated_tuberculosis", "had_tuberculosis_this_year"])

In [10]:
# Map some rows to boolean
df["had_covid_this_year"] = (
    df["had_covid_this_year"].map({"Yes": True, "No": False}).astype("boolean")
)

df["is_vaccinated_influenza"] = (
    df["is_vaccinated_influenza"].map({"Yes": True, "No": False}).astype("boolean")
)

df["had_influenza_this_year"] = (
    df["had_influenza_this_year"].map({"Yes": True, "No": False}).astype("boolean")
)

In [11]:
# Drop rows where any of the essential fields is missing
df = df.dropna(subset=[
    "age",
    "gender",
    "region",
    "blood_group",
    "igm_level",
    "igg_level",
    "had_covid_this_year",
    "is_vaccinated_influenza",
    "had_influenza_this_year",
])

In [12]:
df.isnull().sum()

date_time                     0
age                           0
gender                        0
region                        0
had_covid_this_year           0
igm_level                     0
igg_level                     0
blood_group                   0
is_vaccinated_influenza       0
had_influenza_this_year       0
max_body_temp              1478
dtype: int64

In [13]:
df["gender"] = df["gender"].map({
    "Female (Жінка)": "Female",
    "Male (Чоловік)": "Male",
})
df["gender"].unique()

array(['Female', 'Male'], dtype=object)

In [14]:
df["max_body_temp"].unique()

array([37. ,  nan, 37.4, 36.8, 38.5, 37.1, 36.7, 38.3, 36.9, 37.5, 38.7,
       38. , 37.2, 38.4, 38.2, 38.6, 37.9, 37.3, 36.6, 38.8, 38.1, 37.6,
       37.8, 37.7])

In [15]:
median = round(df["max_body_temp"].median(), 1)
df["max_body_temp"] = df["max_body_temp"].fillna(median)

In [16]:
df.isnull().sum()

date_time                  0
age                        0
gender                     0
region                     0
had_covid_this_year        0
igm_level                  0
igg_level                  0
blood_group                0
is_vaccinated_influenza    0
had_influenza_this_year    0
max_body_temp              0
dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2328 entries, 1 to 3307
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date_time                2328 non-null   datetime64[ns]
 1   age                      2328 non-null   object        
 2   gender                   2328 non-null   object        
 3   region                   2328 non-null   object        
 4   had_covid_this_year      2328 non-null   boolean       
 5   igm_level                2328 non-null   object        
 6   igg_level                2328 non-null   object        
 7   blood_group              2328 non-null   float64       
 8   is_vaccinated_influenza  2328 non-null   boolean       
 9   had_influenza_this_year  2328 non-null   boolean       
 10  max_body_temp            2328 non-null   float64       
dtypes: boolean(3), datetime64[ns](1), float64(2), object(5)
memory usage: 177.3+ KB


In [18]:
df["age"].unique()

array(['23-40', '40-65', '16-22', '>65', '0-15'], dtype=object)

In [19]:
df["age"] = df["age"].astype("category")

In [20]:
df["gender"].unique()

array(['Female', 'Male'], dtype=object)

In [21]:
df["gender"] = df["gender"].astype("category")

In [22]:
df["region"].unique()

array(['Ukraine, Chernivtsi', 'Ukraine, Lviv (Львів)', 'Germany',
       'Germany (Німеччина)', 'Other (Інше)',
       'Ukraine, Chernivtsi (Чернівці)'], dtype=object)

In [23]:
df["igm_level"].unique()

array(['>1.1 (positive/позитивний)', '<0.9 (negative/негативний)',
       '0.9-1.1 (indefinable/невизначений)'], dtype=object)

In [24]:
df["igm_level"] = df["igm_level"].astype("category")

In [25]:
df["igg_level"].unique()

array(['0.9-1.1 (indefinable/невизначений)', '<0.9 (negative/негативний)',
       '>1.1 (positive/позитивний)', '09'], dtype=object)

In [26]:
df["igg_level"] = df["igg_level"].astype("category")

In [27]:
df["blood_group"].unique()

array([2., 1., 3., 4.])

In [28]:
df["blood_group"] = (
    df["blood_group"]
    .astype("Int64")
    .astype("string")
    .astype("category")
)

In [29]:
df.sort_values(
    by=["age", "is_vaccinated_influenza"],
    ascending=[False, True]
).head(5)

,date_time,age,gender,region,had_covid_this_year,igm_level,igg_level,blood_group,is_vaccinated_influenza,had_influenza_this_year,max_body_temp
11,2020-09-20 11:23:15,>65,Female,"Ukraine, Lviv (Львів)",False,<0.9 (negative/негативний),<0.9 (negative/негативний),1,False,False,37.7
22,2020-09-20 11:26:01,>65,Female,"Ukraine, Lviv (Львів)",True,>1.1 (positive/позитивний),<0.9 (negative/негативний),1,False,False,37.1
24,2020-09-20 11:26:01,>65,Male,"Ukraine, Lviv (Львів)",True,>1.1 (positive/позитивний),<0.9 (negative/негативний),2,False,False,37.4
46,2020-09-22 17:35:09,>65,Female,"Ukraine, Lviv (Львів)",False,<0.9 (negative/негативний),<0.9 (negative/негативний),1,False,False,37.7
113,2020-09-23 09:26:30,>65,Male,"Ukraine, Lviv (Львів)",False,<0.9 (negative/негативний),<0.9 (negative/негативний),1,False,False,37.7


In [30]:
df.loc[
    df["had_covid_this_year"] & df["had_influenza_this_year"],
    "max_body_temp"
].mean()

np.float64(37.71666666666667)

In [31]:
df["had_influenza_this_year"].value_counts()

had_influenza_this_year
False    1942
True      386
Name: count, dtype: Int64

In [32]:
# Distribution of influenza cases by gender
df.groupby(["gender", "had_influenza_this_year"]).size().unstack().plot(
    kind="bar",
    stacked=True
)

plt.title("Distribution of influenza cases by gender")
plt.ylabel("Count")
plt.xlabel("Gender")
plt.legend(title="Had influenza this year")
plt.show()

In [35]:
# Distribution of Covid cases by influenza vaccination
df.groupby(["is_vaccinated_influenza", "had_covid_this_year"]).size().unstack().plot(
    kind="bar",
    stacked=True
)

plt.title("Covid cases vs Influenza vaccination")
plt.ylabel("Count")
plt.xlabel("Vaccinated against influenza")
plt.legend(title="Had Covid this year")
plt.show()